In [1]:
import pandas as pd

file_path = 'SRA_data.csv'
columns_to_keep = ['Run','Experiment','TaxID', 'spots', 'size_MB', 'ScientificName','Platform', 'Model', 'ReleaseDate', 'CenterName', 'Country','bases','LibrarySource','LibraryStrategy']
data = pd.read_csv(file_path, usecols=columns_to_keep,dtype=str)
columns_to_convert = ['size_MB', 'bases', 'spots',  'TaxID', 'ReleaseDate']
data = data[data['size_MB'] != 'size_MB']
columns_str = ['Run', 'Experiment', 'ScientificName', 'Platform', 'Model', 'CenterName', 'Country','LibrarySource','LibraryStrategy']
data[columns_str] = data[columns_str].replace([None,'','nan'], pd.NA).fillna('Unknown')
data[columns_str] = data[columns_str].astype(str)
data[columns_to_convert] = data[columns_to_convert].replace([None,'','nan'], pd.NA).fillna(0)
data[columns_to_convert] = data[columns_to_convert].astype(float)
data = data.apply(lambda x: str(x)[:-1] if isinstance(x, str) and x.endswith(' ') else x)
data = data.apply(lambda x: str(x)[:-1] if isinstance(x, str) and x.endswith(' ') else x)
data = data.drop_duplicates(subset=['Run','Experiment'], keep='first')
data[columns_to_convert] = data[columns_to_convert].astype('int64')
filtered_data = data[data['Platform'] != 'Unknown']
grouped_data = filtered_data.groupby('Model')['Platform'].unique()
data = pd.merge(data, grouped_data, on='Model', how='inner')
data = data.drop('Platform_x', axis=1)
data = data.rename(columns={'Platform_y': 'Platform'})
data['Platform'] = data['Platform'].astype(str).str.replace("[", '')
data['Platform'] = data['Platform'].astype(str).str.replace("]", '')
data['Platform'] = data['Platform'].astype(str).str.replace("'", '')
data['Platform'] = data['Platform'].astype(str).str.replace('"', '')
data = data.drop_duplicates(subset=['Run','Experiment'], keep='first')

data.to_csv('SRA_graf_inf.csv', index=False)


In [6]:
import pandas as pd

file_path = 'SRA_data.csv'
columns_to_keep = ['Run','Experiment',"TaxID", "Disease",  "Tumor", "Sex"]
data = pd.read_csv(file_path, usecols=columns_to_keep,dtype=str)
data.replace(['None', '0', '','nan','OTHER','Other','Unknown',0], pd.NA, inplace=True)
print(data.shape)
for column in data.columns:
    nan_count = data[column].isna().sum()
    print(f'Количество NaN значений в колонке "{column}": {nan_count}')


(28124150, 6)
Количество NaN значений в колонке "Run": 0
Количество NaN значений в колонке "Experiment": 0
Количество NaN значений в колонке "TaxID": 1469433
Количество NaN значений в колонке "Disease": 28121593
Количество NaN значений в колонке "Sex": 24299228
Количество NaN значений в колонке "Tumor": 1468054


In [5]:
import pandas as pd

file_path = 'SRA_data.csv'
columns_to_keep = ['Run','Experiment',"TaxID", "Disease",  "Tumor", "Sex"]
data = pd.read_csv(file_path, usecols=columns_to_keep,dtype=str)
data=data[data['TaxID']== '9606' ]
data.replace(['None', '0', '','nan','OTHER','Other','Unknown',0,'no'], pd.NA, inplace=True)
print(data.shape)
for column in data.columns:
    nan_count = data[column].isna().sum()
    print(f'Количество NaN значений в колонке "{column}": {nan_count}')


(4866940, 6)
Количество NaN значений в колонке "Run": 0
Количество NaN значений в колонке "Experiment": 0
Количество NaN значений в колонке "TaxID": 0
Количество NaN значений в колонке "Disease": 4864390
Количество NaN значений в колонке "Sex": 2720718
Количество NaN значений в колонке "Tumor": 4725271


In [ ]:
import pandas as pd

file_path = 'SRA_data.csv'
columns_to_keep = ['Run','Experiment','ScientificName','TaxID']
data = pd.read_csv(file_path, usecols=columns_to_keep,dtype=str)
filtered_data = data[~data['ScientificName'].str.contains('metagenom')]

nan_count = filtered_data['TaxID'].isna().sum()

print("Количество NaN значений в колонке 'taxid':", nan_count)
print(data.shape)

empty_count = (data['LibraryStrategy'] == 'OTHER').sum()
print(empty_count)
total_rows = len(data)

empty_percentage = (empty_count / total_rows) * 100
print(empty_percentage)




In [ ]:
import pandas as pd

file_path = 'SRA_graf_inf.csv'
data = pd.read_csv(file_path, dtype=str)

def get_first_two_words(text):
    if isinstance(text, str) and 'metagenome' in text.lower():
        return "metagenome"
    elif isinstance(text, str) :
        words = text.split() 
        return ' '.join(words[:2]) if len(words) >= 2 else text
    else: return ""
def get_first_words(text):
    if isinstance(text, str) and 'metagenome' in text.lower():
        return "metagenome"
    elif isinstance(text, str) :
        words = text.split() 
        return ' '.join(words[:1]) if len(words) >= 1 else text
    else: return ""
data_words = data['ScientificName'].apply(get_first_two_words)

counts = data_words.value_counts()

counts_df = counts.reset_index()
counts_df.columns = ['SC Combination', 'Count']


counts_df.loc[counts_df['Count'] < 10000, 'SC Combination'] = counts_df.loc[counts_df['Count'] < 10000, 'SC Combination'].apply(get_first_words)
counts_df = counts_df.groupby('SC Combination')['Count'].sum().reset_index()
counts_df  = counts_df.sort_values(by='Count', ascending=False).reset_index(drop=True)
sum_counts = counts_df.loc[counts_df['Count'] > 10000, 'Count'].sum()

print(f"Сумма 'Count' для строк, где 'Count' > 10000: {sum_counts}") 
sum_counts = counts_df.loc[counts_df['Count'] > 1000, 'Count'].sum()

print(f"Сумма 'Count' для строк, где 'Count' > 1000: {sum_counts}")

sum_counts = counts_df.loc[counts_df['Count'] > 100, 'Count'].sum()

print(f"Сумма 'Count' для строк, где 'Count' > 100: {sum_counts}")

sum_counts = counts_df.loc[counts_df['Count'] > 0, 'Count'].sum()
print(f"Сумма 'Count' для строк, где 'Count' > 0: {sum_counts}")

counts_df.to_csv('SC_counts.csv', index=False, encoding='utf-8')

data['SC'] = data['ScientificName'].apply(get_first_two_words)

counts = data_words.value_counts()

counts_df = counts.reset_index()
counts_df.columns = ['SC', 'Count']
data=pd.merge(data,counts_df,on='SC',how='inner')

data.loc[data['Count'] < 10000, 'SC'] = data.loc[data['Count'] < 10000, 'SC'].apply(get_first_words)
data = data.drop('Count', axis=1)
data.to_csv('data_organism_class_tmp.csv', index=False, encoding='utf-8')


Сумма 'Count' для строк, где 'Count' > 10000: 25725510
Сумма 'Count' для строк, где 'Count' > 1000: 27146665
Сумма 'Count' для строк, где 'Count' > 100: 27852333
Сумма 'Count' для строк, где 'Count' > 0: 28124143


In [1]:
import pandas as pd

columns_to_keep = ['Run','Experiment','ScientificName','size_MB','SC']
data = pd.read_csv('data_organism_class_tmp.csv', usecols=columns_to_keep,dtype=str)#tmp_csv

data_class_org = pd.read_csv('class_org.csv',dtype=str)
data['ScientificName_TMP'] = data['SC'].copy()
data['SC'] = data['SC'].str.lower().str.replace(' ', '')
data_class_org['SC'] = data_class_org['SC'].str.lower().str.replace(' ', '')

data_main=pd.merge(data,data_class_org,how='outer',on='SC').drop_duplicates()
data_main['Class_organism'] = data_main['Class_organism'].fillna('Unknown')

data_main['SC'] = data_main['ScientificName_TMP'].copy()
data_main = data_main.drop('ScientificName_TMP', axis=1)

print(data_main.shape)
data_main.to_csv('data_organism_class.csv', index=False, encoding='utf-8')
print(data_main['Class_organism'].value_counts())


(28124148, 6)
Class_organism
Vertebrates      9198075
Viruses          6793311
Metagenome       5691523
Bacteria         2610942
Plants           1728005
Unknown           758286
Invertebrates     651186
Fungi             348098
Protists          338341
Archaea             6381
Name: count, dtype: int64


In [ ]:
import pandas as pd

data = pd.read_csv('data_organism_class.csv',dtype=str)
data_metagenome_class = pd.read_csv('count_ScientificName_metagenome.csv',dtype=str)
data_main=pd.merge(data,data_metagenome_class,how='outer',on='ScientificName').drop_duplicates()
data_main = data_main.drop('Count', axis=1)
data_main.to_csv('data_organism_class_plus_metagenome.csv', index=False, encoding='utf-8')

print(data_main.shape)
print(data_main['Class_metagenome'].value_counts())
print(data_main[data_main['Class_organism'] == 'Metagenome']['ScientificName'].value_counts())

(28124150, 7)
Class_metagenome
Human                    1807297
Environment              1695369
Animal                    806755
metagenome                538040
Plant                     464550
Artificial ecosystems     379415
Name: count, dtype: int64
ScientificName
soil metagenome                   777926
human gut metagenome              694138
metagenome                        529976
gut metagenome                    364223
human metagenome                  240470
                                   ...  
iron metagenome                        2
moonmilk metagenome                    2
desalination cell metagenome           1
metal metagenome                       1
chemical production metagenome         1
Name: count, Length: 351, dtype: int64


In [ ]:
import pandas as pd

file_path = 'SRA_graf_inf.csv'
data = pd.read_csv(file_path)
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatter
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

replacement_dict = {
    'Western Sahara': 'W. Sahara',
    'Democratic Republic of the Congo': 'Dem. Rep. Congo',
    'French Southern and Antarctic Lands': 'Fr. S. Antarctic Lands',
    'East Timor': 'Timor-Leste',
    'Eswatini': 'eSwatini',
    'Equatorial Guinea': 'Eq. Guinea',
    'South Korea': 'Korea, South',
    'North Korea': 'Korea, North',
    'Czech Republic': 'Czechia',
    'Macedonia': 'North Macedonia',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Northern Cyprus': 'N. Cyprus',
    'South Sudan': 'S. Sudan',
    "Côte d'Ivoire": 'Ivory Coast',
    'United States': 'United States of America',
    'United Kingdom': 'United Kingdom',
    'French Southern and Antarctic Lands': 'Fr. S. Antarctic Lands',
    'East Timor': 'Timor-Leste',
    'Korea, South': 'South Korea',
    'Korea, North': 'North Korea',
    'Czechia': 'Czech Republic',
    'North Macedonia': 'Macedonia',
    'Bosnia and Herz.': 'Bosnia and Herzegovina',
    'N. Cyprus': 'Northern Cyprus',
    'S. Sudan': 'South Sudan',
}

reversed_dict = {v: k for k, v in replacement_dict.items()}

country_freq = data['Country'].value_counts().reset_index()
country_freq.columns = ['name', 'Frequency']
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world['name'] = world['name'].replace(reversed_dict)

unique_continents = world['name'].unique()
world = pd.merge(world, country_freq, on='name', how='left')
world_regions = world.drop_duplicates(subset='name', keep='first')
world_regions = world_regions[['name', 'continent']]  # Use double square brackets or pass a list of column names
world_regions.to_csv('world_regions.csv', index=False)


In [ ]:
import csv
csv.field_size_limit(100000000)

with open('SRA_data.csv', mode='r', newline='', encoding='utf-8') as input_file:
    with open('output.csv', mode='w', newline='', encoding='utf-8') as output_file:
        reader = csv.reader(input_file)
        writer = csv.writer(output_file)
        
        headers = next(reader)
        writer.writerow(headers)
        
        for row in reader:
            if row[23] == 'SYNTHETIC': 
                writer.writerow(row)

In [1]:

import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
file_path = 'SRA_data.csv'
columns_to_keep = ['Run','Country']
data = pd.read_csv(file_path, usecols=columns_to_keep)



In [ ]:

from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatter
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap


norm = LogNorm(vmin=1000, vmax=12000000)

replacement_dict = {
    'Western Sahara': 'W. Sahara',
    'Democratic Republic of the Congo': 'Dem. Rep. Congo',
    'French Southern and Antarctic Lands': 'Fr. S. Antarctic Lands',
    'East Timor': 'Timor-Leste',
    'Eswatini': 'eSwatini',
    'Equatorial Guinea': 'Eq. Guinea',
    'South Korea': 'Korea, South',
    'North Korea': 'Korea, North',
    'Czech Republic': 'Czechia',
    'Macedonia': 'North Macedonia',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Northern Cyprus': 'N. Cyprus',
    'South Sudan': 'S. Sudan',
    "Côte d'Ivoire": 'Ivory Coast',
    'United States': 'United States of America',
    'United Kingdom': 'United Kingdom',
    'French Southern and Antarctic Lands': 'Fr. S. Antarctic Lands',
    'East Timor': 'Timor-Leste',
    'Korea, South': 'South Korea',
    'Korea, North': 'North Korea',
    'Czechia': 'Czech Republic',
    'North Macedonia': 'Macedonia',
    'Bosnia and Herz.': 'Bosnia and Herzegovina',
    'N. Cyprus': 'Northern Cyprus',
    'S. Sudan': 'South Sudan',
}

data['Country'] = data['Country'].replace(replacement_dict)

country_freq = data['Country'].value_counts().reset_index()
country_freq.columns = ['name', 'Frequency']
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
unique_continents = world['name'].unique()

world = pd.merge(world, country_freq, on='name', how='left')

world['Frequency'].fillna(1, inplace=True)
fig, ax = plt.subplots(1, 1, figsize=(15, 7), dpi=700)
colors = [ (1, 1, 0.9), (1, 0.5, 0), (1, 0, 0)]  
positions = [0, 0.33, 0.66, 1]

cmap = LinearSegmentedColormap.from_list('custom_gradient', colors, N=256)

plot = world.plot(column='Frequency', cmap=cmap, alpha=1 ,linewidth=0.8, ax=ax, edgecolor='0.8', legend=False, norm=norm)
formatter = LogFormatter(10, labelOnlyBase=False)
cbar = fig.colorbar(plot.get_children()[0], ax=ax, format=formatter, shrink=0.8)


cbar.set_ticks([1000,10000, 100000, 1000000, 4000000, 12000000])
cbar.set_ticklabels(['<1k','10k', '100k', '1M', '4M', '>12M'])

ax.set_title('Heatmap by "Run" quantity')
ax.set_axis_off()

plt.show()

In [ ]:

import csv
import random
import pandas as pd

csv.field_size_limit(100000000)

input_file_path = 'SRA_data.csv'
output_file_path = 'small_SRA_data.csv'

def file_len(fname):
    with open(fname,newline='', encoding='utf-8') as f:
        for i, l in enumerate(f):
            pass
    return i + 1

n = 100000
total_lines = file_len(input_file_path) - 1
selected_lines = sorted(random.sample(range(1, total_lines + 1), n))

with open(input_file_path, 'r', newline='', encoding='utf-8') as infile, \
        open(output_file_path, 'w', newline='', encoding='utf-8') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    writer.writerow(next(reader))

    for line_number, row in enumerate(reader, start=1):
        if line_number == selected_lines[0]:
            writer.writerow(row)
            selected_lines.pop(0)
            if not selected_lines:
                break

print("Выборка завершена и сохранена в", output_file_path)


In [ ]:
import pandas as pd

file_path1 = 'SRA_data.csv'
file_path2 = 'SRA_fulldata.csv'
df1 = pd.read_csv(file_path1, usecols=['Experiment'])
df2 = pd.read_csv(file_path2, usecols=['Experiment'])

unique_values_df1 = df1['Experiment'].nunique()
unique_values_df2 = df2['Experiment'].nunique()


unique_values_only_in_df1 = df1[~df1['Experiment'].isin(df2['Experiment'])]
unique_values_only_in_df2 = df2[~df2['Experiment'].isin(df1['Experiment'])]

print(f'Уникальных значений в df1, которых нет в df2: {len(unique_values_only_in_df1)}')
print(f'Уникальных значений в df2, которых нет в df1: {len(unique_values_only_in_df2)}')




Уникальных значений в df1, которых нет в df2: 1467839
Уникальных значений в df2, которых нет в df1: 0


In [4]:
print(df1.drop_duplicates().shape)
print(df2.drop_duplicates().shape)

(26767311, 1)
(25299478, 1)


In [3]:
import pandas as pd
file_path = 'small_SRA_data.csv'
small_SRA_data = pd.read_csv(file_path,dtype=str)
file_path = 'Run_Score.csv'
Completeness_Data = pd.read_csv(file_path,dtype=str)
merged_data = pd.merge(small_SRA_data, Completeness_Data, on='Run', how='inner')
merged_data.to_csv('small_SRA_data.csv', index=False)

In [1]:

import csv
import random
csv.field_size_limit(100000000)
input_file_path = 'SRA_data.csv'
output_file_path = 'Completeness_Data.csv'
def check_and_replace(value):
    if value in ['nan', None, '', 'Unknown', 'Other', 'other', '0', '-1',0,-1]:
        return 0
    else:
        return 1
with open(input_file_path, 'r', newline='', encoding='utf-8') as input_file:
    with open(output_file_path, 'w', newline='', encoding='utf-8') as output_file:
        reader = csv.reader(input_file)
        writer = csv.writer(output_file)
        header = next(reader)
        writer.writerow(header)
        for row in reader:
            first_column = row[0]          
            processed_row = [first_column] + [check_and_replace(value) for value in row[1:]]
            writer.writerow(processed_row)



In [1]:
import pandas as pd

file_path = 'Completeness_data.csv'
data = pd.read_csv(file_path,dtype=str)
data.drop(columns=['Tumor'], inplace=True)
data.drop(columns=['Affection_Status'], inplace=True)
data.drop(columns=['Analyte_Type'], inplace=True)
data.drop(columns=['AssemblyName'], inplace=True)
data.drop(columns=['Body_Site'], inplace=True)
data.drop(columns=['Consent'], inplace=True)
data.drop(columns=['InsertDev'], inplace=True)
data.drop(columns=['InsertSize'], inplace=True)
data.drop(columns=['Country'], inplace=True)
data.drop(columns=['Study_Pubmed_id'], inplace=True)
data.drop(columns=['Disease'], inplace=True)
print(data.shape)
def calculate_score(row):
    score = int(sum([float(x) for x in row[1:]]) * 3.125)
    return score

data['Score'] = data.apply(calculate_score, axis=1)
data[['Run', 'Score']].to_csv('Run_Score.csv', index=False)

(28124150, 32)
